In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Easy
How many unique customers are in the dataset?

What are the different categories of products available? How many unique categories are there?

Which payment method is the most popular? How many times was it used?

# Medium
What are the three most popular categories, by total sales?

What are the total sales attributed to customers over the age of 45?

How is the data distributed across different invoice dates? Are there any seasonal trends or patterns? (Use a graph for this.)


# Hard
Create a pivot table showing the breakdown of sales across these dimensions, in this order: category, decade age range (e.g., 10-19, 20-29, and so forth).